In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# move to the target folder and listing the content
%cd /content/drive/MyDrive/Machine_Learning/
!ls

/content/drive/MyDrive/Machine_Learning
1_image_facial_recognition_basic.ipynb
2_live_webcam_facial_landmarks_CNN.ipynb
3_live_webcam_facial_landmarks_drowsiness_detection_CNN.ipynb
3_live_webcam_facial_landmarks_drowsiness_detection_MTCNN.ipynb
image_facial_emotion_recognition_MTCNN.ipynb
Images
Models
sound


In [ ]:
!pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=9bab194952aa27cdfd44c0078c0d10826b87712b14e2b62eed936daa1e4743cf
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [ ]:
!pip install mtcnn-opencv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.4 MB/s eta 0:00:00


In [ ]:
# import the necessary packages
from IPython.display import display, Javascript, Image, Audio
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from google.colab.patches import cv2_imshow
from mtcnn_cv2 import MTCNN
from imutils import face_utils
import dlib
import cv2
import numpy as np
import playsound
import PIL
import io


In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes


# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# compute and return the euclidean distance between the two points
def euclidean_dist(ptA, ptB):
	return np.linalg.norm(ptA - ptB)

# compute the euclidean distances between the two sets of
# vertical eye landmarks (x, y)-coordinates
def eye_aspect_ratio(eye):
	A = euclidean_dist(eye[1], eye[5])
	B = euclidean_dist(eye[2], eye[4])
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = euclidean_dist(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
	# return the eye aspect ratio
	return ear

In [ ]:
# function to predic faces using the MTCCN model
def predict_mtcnn(image, predictor, bbox_array):
  global counter
  global alarm_on

  # convert image from RGB to Grayscale
  image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # define the detector
  detector = MTCNN()

  # detect faces in the image
  faces = detector.detect_faces(image)

  for result in faces:
    # get coordinates from face
    x, y, w, h = result['box']

    # create the face bounding box
    #bbox_array = cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy array
    rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))

    shape = predictor(image_gray, rect)
    shape = face_utils.shape_to_np(shape)

    # extract the left and right eye coordinates, then use the
    # coordinates to compute the eye aspect ratio for both eyes
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    # average the eye aspect ratio together for both eyes
    ear = (leftEAR + rightEAR) / 2.0

    # compute the convex hull for the left and right eye, then
    # visualize each of the eyes
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    bbox_array = cv2.drawContours(bbox_array, [leftEyeHull], -1, (0, 255, 0), 1)
    bbox_array = cv2.drawContours(bbox_array, [rightEyeHull], -1, (0, 255, 0), 1)

    # check to see if the eye aspect ratio is below the blink
    # threshold, and if so, increment the blink frame counter
    if ear < EYE_AR_THRESH:
      counter += 1
      # if the eyes were closed for a sufficient number of
      # frames, then sound the alarm
      if counter >= EYE_AR_CONSEC_FRAMES:
        # if the alarm is not on, turn it on
        if not alarm_on:
          alarm_on = True
          wn = Audio('sound/alarm.wav', autoplay=True)
          display(wn)

        # draw an alarm text on the frame
        bbox_array = cv2.putText(bbox_array, "DROWSINESS ALERT!", (10, 30),
          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    # otherwise, the eye aspect ratio is not below the blink
    # threshold, so reset the counter and alarm
    else:
      counter = 0
      alarm_on = False

    # draw the computed eye aspect ratio on the frame to help
		# with debugging and setting the correct eye aspect ratio
		# thresholds and frame counters
    bbox_array = cv2.putText(bbox_array, "EAR: {:.3f}".format(ear), (300, 30),
			   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0 ), 2)

    return bbox_array

In [ ]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 16
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
counter = 0
alarm_on = False

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
predictor = dlib.shape_predictor("Models/shape_predictor_68_face_landmarks.dat")

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
# initialize the video stream and pointer to output video file, then
# allow the camera sensor to warm up
print("Starting video stream...")
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''

# loop over frames from the video file stream
while True:
      js_reply = video_frame(label_html, bbox)

      if not js_reply:
        break

      # convert JS response to OpenCV Image
      image_frame = js_to_image(js_reply["img"])

      image_frame = cv2.cvtColor(image_frame, cv2.COLOR_BGR2RGB)

      # create transparent overlay for bounding box
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      # predict face in the current frame
      bbox_array = predict_mtcnn(image_frame, predictor, bbox_array)

      bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
      # convert overlay of bbox into bytes
      bbox_bytes = bbox_to_bytes(bbox_array)
      # update bbox so next frame gets new overlay
      bbox = bbox_bytes

# do a bit of cleanup
cv2.destroyAllWindows()

Starting video stream...


<IPython.core.display.Javascript object>